In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00


In [3]:
!pip install torch transformers nltk bert-score matplotlib tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [4]:
from datasets import load_dataset
train_data = load_dataset("wmt16","de-en", split="train[:50000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from transformers import EncoderDecoderModel, EncoderDecoderConfig, BertTokenizer
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score
from bert_score import score as bert_score
from torch.nn.utils.rnn import pad_sequence
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')

# Define dataset class
class TranslationDataset(Dataset):
    def __init__(self, data, tokenizer_de, tokenizer_en):
        self.data = data
        self.tokenizer_de = tokenizer_de
        self.tokenizer_en = tokenizer_en

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        translation = self.data[idx]['translation']  # Access the 'translation' key
        src_text = translation['de']  # Access the 'de' key within 'translation'
        tgt_text = translation['en']  # Access the 'en' key within 'translation'
        src_tokens = self.tokenizer_de(src_text, padding=True, truncation=True, return_tensors='pt')['input_ids']
        tgt_tokens = self.tokenizer_en(tgt_text, padding=True, truncation=True, return_tensors='pt')['input_ids']
        return {'src_tokens': src_tokens.squeeze(), 'tgt_tokens': tgt_tokens.squeeze()}

def collate_fn(batch):
    # Sort batch by sequence length for efficient padding
        batch = sorted(batch, key=lambda x: x['src_tokens'].shape[0], reverse=True)

        # Pad sequences to the length of the longest sequence in the batch
        src_tokens = pad_sequence([x['src_tokens'] for x in batch], batch_first=True, padding_value=tokenizer_de.pad_token_id)
        tgt_tokens = pad_sequence([x['tgt_tokens'] for x in batch], batch_first=True, padding_value=tokenizer_en.pad_token_id)

        return {'src_tokens': src_tokens, 'tgt_tokens': tgt_tokens}

# Define model configuration
config = EncoderDecoderConfig.from_encoder_decoder_configs(
    EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-german-cased', 'bert-base-uncased').config.encoder,
    EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-german-cased', 'bert-base-uncased').config.decoder
)

# Initialize tokenizer
tokenizer_de = BertTokenizer.from_pretrained('bert-base-german-cased')
tokenizer_en = BertTokenizer.from_pretrained('bert-base-uncased')

# Define model
model = EncoderDecoderModel(config)

# Define optimizer and loss
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Load data
train_data = load_dataset("wmt16","de-en", split="train[:50000]")  # Load training data
valid_data = load_dataset("wmt16","de-en", split="validation")  # Load validation data
test_data = load_dataset("wmt16","de-en", split="test")  # Load test data

# Initialize datasets and dataloaders
train_dataset = TranslationDataset(train_data, tokenizer_de, tokenizer_en)
valid_dataset = TranslationDataset(valid_data, tokenizer_de, tokenizer_en)
test_dataset = TranslationDataset(test_data, tokenizer_de, tokenizer_en)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

# Implement gradient accumulation
accumulation_steps = 4  # Accumulate gradients over 4 steps

# Training loop
'''def train_epoch(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        src_tokens = batch['src_tokens'].to(device)
        tgt_tokens = batch['tgt_tokens'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids=src_tokens, decoder_input_ids=tgt_tokens[:, :-1], labels=tgt_tokens[:, 1:])
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), tgt_tokens[:, 1:].view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)'''

def train_epoch(model, dataloader, criterion, optimizer, accumulation_steps, device):
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(tqdm(dataloader, desc="Training")):  # Add batch_idx here
        src_tokens = batch['src_tokens'].to(device)
        tgt_tokens = batch['tgt_tokens'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids=src_tokens, decoder_input_ids=tgt_tokens[:, :-1], labels=tgt_tokens[:, 1:].contiguous())
        logits_shape = outputs.logits.shape
        labels_shape = tgt_tokens[:, 1:].shape
        #print("Logits shape:", logits_shape)
        #print("Labels shape:", labels_shape)
        loss = criterion(outputs.logits.reshape(-1, outputs.logits.size(-1)), tgt_tokens[:, 1:].reshape(-1))
        loss = loss / accumulation_steps  # Normalize the loss
        loss.backward()  # Backpropagate the gradients
        if (batch_idx + 1) % accumulation_steps == 0:  # Only update the weights every `accumulation_steps` batches
            optimizer.step()
            optimizer.zero_grad()
        total_loss += loss.item()
    return total_loss / len(dataloader)


# Evaluation function
# Move model to the same device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def evaluate(model, dataloader, device):
    model.eval()
    predictions = []
    targets = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluation"):
            src_tokens = batch['src_tokens'].to(device)
            tgt_tokens = batch['tgt_tokens'].to(device)

            # Generate predictions
            generated = model.generate(src_tokens, decoder_start_token_id=tokenizer_en.cls_token_id)
            predictions.extend(generated)
            targets.extend(tgt_tokens[:, 1:].cpu().numpy())
    predictions = [tokenizer_en.decode(ids, skip_special_tokens=True) for ids in predictions]
    targets = [tokenizer_en.decode(ids, skip_special_tokens=True) for ids in targets]
    return predictions, targets




# Training loop
num_epochs = 5
train_losses = []
valid_losses = []

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, accumulation_steps,device)
    valid_loss = evaluate(model, valid_loader,device)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

# Save model checkpoint
torch.save(model.state_dict(), 'model_checkpoint.pth')

# Evaluation
test_predictions, test_targets = evaluate(model, test_loader, device)
tokenized_reference = [word_tokenize(ref) for ref in test_targets]

# Calculate evaluation metrics
bleu_score = corpus_bleu([[tgt.split()] for tgt in test_targets], [pred.split() for pred in test_predictions])
meteor_score_avg = meteor_score(tokenized_reference, test_predictions)
bert_score_p, bert_score_r, bert_score_f1 = bert_score(test_predictions, test_targets, lang='en', model_type='bert-base-uncased', nthreads=4)

# Print evaluation metrics
print("BLEU Score:", bleu_score)
print("METEOR Score:", meteor_score_avg)
print("BERTScore Precision:", bert_score_p.mean())
print("BERTScore Recall:", bert_score_r.mean())
print("BERTScore F1:", bert_score_f1.mean())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Training:   0%|          | 0/3125 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:636: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Training:  53%|█████▎    | 1644/3125 [15:12<13:41,  1.80it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 39.06 MiB is free. Process 2255 has 14.71 GiB memory in use. Of the allocated memory 12.99 GiB is allocated by PyTorch, and 1.58 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)